In [1]:
import napari
import numpy as np
import matplotlib.pyplot as plt
import z5py
from pathlib import Path

from cryofib.affine_utils import parse_points

In [2]:
def print_key_tree(f: z5py.File):
    print(f"Key structure of z5 file {f.filename}")
    f.visititems(lambda name, obj: print(name))


def read_volume(f: z5py.File, key: str, roi: np.lib.index_tricks.IndexExpression = np.s_[:]):
    try:
        ds = f[key]
    except KeyError:
        print(f"No key {key} in file {f.filename}")
        print_key_tree(f)
        return None
        
    ds.n_threads = 8
    print(f"Reading roi {roi} of volume {key} from {f.filename}")
    vol = ds[roi]
    print(f"Read volume with shape {vol.shape}, data type {vol.dtype}")
    
    return vol

## View n5 datasets

In [3]:
scratch_dir = Path("/scratch/buglakova/data/cryofib/registration_fluo/F107_A2_3dclem")
fluo_dir = scratch_dir / "fluo"
fluo_n5_path = fluo_dir / "fluo.n5"
segm_em_nuclei_dir = scratch_dir / "segm_em_nuclei"
segm_em_nuclei_path = segm_em_nuclei_dir / "em_nuclei.n5"


em_dir = scratch_dir / "em"
em_n5_path = em_dir / "em.n5"

f_fluo = z5py.File(fluo_n5_path, "r")
f_em = z5py.File(em_n5_path, "r")
f_segm_em_nuclei = z5py.File(segm_em_nuclei_path, "r")

In [4]:
output_dir = Path("/scratch/buglakova/data/cryofib/segm_fluo/cellpose")
output_n5_path = output_dir / "cellpose_2D.n5"
f_out = z5py.File(output_n5_path, "r")

In [5]:
alignment_points_path = scratch_dir / "Probe_coordinates_for_Alyona.txt"

## Read volumes

In [6]:
roi = np.s_[:]
fluo = read_volume(f_fluo, "raw", roi)

Reading roi slice(None, None, None) of volume raw from /scratch/buglakova/data/cryofib/registration_fluo/F107_A2_3dclem/fluo/fluo.n5
Read volume with shape (55, 3343, 1482), data type uint8


In [7]:
roi = np.s_[:]
segm = read_volume(f_out, "cellpose", roi)
segm_background = read_volume(f_out, "cellpose_background", roi)

Reading roi slice(None, None, None) of volume cellpose from /scratch/buglakova/data/cryofib/segm_fluo/cellpose/cellpose_2D.n5
Read volume with shape (55, 3343, 1482), data type uint16
Reading roi slice(None, None, None) of volume cellpose_background from /scratch/buglakova/data/cryofib/segm_fluo/cellpose/cellpose_2D.n5
Read volume with shape (55, 3343, 1482), data type uint16


In [8]:
em = read_volume(f_em, "raw", roi)

Reading roi slice(None, None, None) of volume raw from /scratch/buglakova/data/cryofib/registration_fluo/F107_A2_3dclem/em/em.n5
Read volume with shape (1410, 1386, 1610), data type uint8


In [9]:
segm_em_nuclei = read_volume(f_segm_em_nuclei, "raw", roi)

Reading roi slice(None, None, None) of volume raw from /scratch/buglakova/data/cryofib/registration_fluo/F107_A2_3dclem/segm_em_nuclei/em_nuclei.n5
Read volume with shape (1410, 1386, 1610), data type uint8


In [10]:
image_names = {"fluo": "HT6_3Dcrop",
                "em": "F107_a2_ali_crop_from70_bin2_ORS-denoised_YZX"}

coords = parse_points(alignment_points_path, image_names)

fluo
{'val': '156', 'x': '458', 'y': '498', 'z': '4'}
{'val': '159', 'x': '462', 'y': '747', 'z': '7'}
{'val': '148', 'x': '431', 'y': '569', 'z': '10'}
{'val': '112', 'x': '445', 'y': '548', 'z': '14'}
{'val': '52', 'x': '553', 'y': '667', 'z': '1'}
{'val': '125', 'x': '433', 'y': '569', 'z': '11'}
em
{'val': '103', 'x': '70', 'y': '890', 'z': '686'}
{'val': '124', 'x': '1443', 'y': '725', 'z': '833'}
{'val': '131', 'x': '478', 'y': '1035', 'z': '1082'}
{'val': '66', 'x': '359', 'y': '933', 'z': '1312'}
{'val': '101', 'x': '959', 'y': '116', 'z': '439'}
{'val': '114', 'x': '478', 'y': '1018', 'z': '1116'}


## Open it in napari

In [12]:
v = napari.Viewer()

### Open untransformed fluorescence

In [17]:
ds = f_fluo["raw"]
resolution = ds.attrs["resolution"]
v.add_image(fluo, scale=resolution)
points_layer = v.add_points(coords["fluo"], name="ref points", scale=resolution, size=10, face_color="green", out_of_slice_display=True)

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/numpy/core/numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [18]:
fluo.shape

(55, 3343, 1482)

### Open untransformed EM

In [19]:
ds = f_em["raw"]
resolution = ds.attrs["resolution"]
v.add_image(em, scale=resolution)
v.add_labels(segm_em_nuclei, scale=resolution)
v.add_points(coords["em"], name="em points", scale=resolution, size=30, face_color="cyan", out_of_slice_display=True)

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/numpy/core/numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


<Points layer 'em points' at 0x7f599521e9e0>

### Open transformed EM

In [24]:
R = [[ 9.99593298e-01, -1.89529264e-03,  2.84542785e-02, -1.54529299e+04],
 [-2.85173298e-02, -6.64340537e-02,  9.97383215e-01,  1.09790052e+05],
 [-0.00000000e+00, -9.97789018e-01, -6.64610835e-02,  1.27570851e+05],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]


In [25]:
ds = f_em["raw"]
resolution = ds.attrs["resolution"]
v.add_image(em, name="em transformed", scale=resolution, affine=R)
v.add_labels(segm_em_nuclei, name="em nuclei transformed", scale=resolution, affine=R)
v.add_points(coords["em"], name="em points transformed", scale=resolution, affine=R, size=30, face_color="red", out_of_slice_display=True)

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being 

<Points layer 'em points transformed [1]' at 0x7f59955f1360>

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being 

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being 

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being 

...

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being 

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being 

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being 

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being 

/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/home/buglakov/miniconda3/envs/napari_env/lib/python3.10/site-packages/napari/layers/base/base.py:747: UserWarning: Non-orthogonal slicing is being 